

# Project: Investigate a Dataset (Analysing TMDB Movie Data )

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

> **TMDB Movie Data**: This data set contains information about 10,000 movies collected from The Movie Database (TMDb), including user ratings and revenue .
>
> **Questions :**
>
> * Whats is the most movie have revenues?
> 
> * what is the relation between geners and revenues ?
> 
> * whitch director contribute in most of them ?
>
> * whitch movie make profit ? max profit? min profit?


In [ ]:
# Use this cell to set up import statements for all of the packages that you
#   plan to use.
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import re
# Remember to include a 'magic word' so that your visualizations are plotted
#   inline with the notebook. See this page for more:
#   http://ipython.readthedocs.io/en/stable/interactive/magics.html


<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you document your steps carefully and justify your cleaning decisions.

### General Properties

In [ ]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.
tmdb_df=pd.read_csv("tmdb-movies.csv")
tmdb_df.head()

In [ ]:
#review columns 
tmdb_df.info()

In [ ]:
# check is there any duplication
tmdb_df.duplicated().value_counts()

In [ ]:
#
tmdb_df.describe()

> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

### Data Cleaning (Replace this with more specific notes!)
> * Remove duplicate Data
> * Some columns we didnot need them [id,imdb_id,popularity]
> * Change data type of release_date  to be date time
> * Change data type of budget_adj and revenue_adj to be Integer as no need for fractions in millions no.
> * Although budget,revenue & budget_adj,revenue_adj has the same information but i will keep both of them as
we need budget,revenue as basic data and need budget_adj,revenue_adj for statistics purboses .
> * Drop all Rows whitch has budget_adj,revenue_adj less than 1000$  as the data seems to be wrong .
> * Some movies run time = 0 but we will do nothing as this column will be removed with rows with 0 budget .

In [ ]:
# Make copy of data to clean
tmdb_df_cleaned=tmdb_df.copy()

In [ ]:
#Drop duplicate values
tmdb_df_cleaned.drop_duplicates(inplace=True)

In [ ]:
#drop columns that i will not use  [id,imdb_id,popularity]
tmdb_df_cleaned.drop(columns=['id','imdb_id','popularity'],inplace=True)

In [ ]:
tmdb_df_cleaned.head(2)

In [ ]:
#Change Data type of  release_date to date time
tmdb_df_cleaned['release_date']=pd.to_datetime(tmdb_df_cleaned['release_date'])
tmdb_df_cleaned.info()

In [ ]:
tmdb_df_cleaned.head(2)

In [ ]:
#Change data type of budget_adj and revenue_adj to be Integer as no need for fractions in millions no.
col=['budget_adj','revenue_adj']
tmdb_df_cleaned[col]=tmdb_df_cleaned[col].astype(int)
tmdb_df_cleaned.info()

In [ ]:
tmdb_df_cleaned.head(2)

In [ ]:
# drop all data >1000$ in budget_adj and revenue_adj
tmdb_df_cleaned=tmdb_df_cleaned[(tmdb_df_cleaned['budget_adj']>1000) & (tmdb_df_cleaned['revenue_adj']>1000)]
tmdb_df_cleaned.info()

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Whats is the The highest revenue and lowest movie has revenues?)

In [ ]:
# movie with highest revenue
tmdb_df_cleaned[tmdb_df_cleaned['revenue_adj']==tmdb_df_cleaned['revenue_adj'].max()]

In [ ]:
tmdb_df_cleaned['revenue_adj'].describe()

In [ ]:
# movie with lowest revenue
tmdb_df_cleaned[tmdb_df_cleaned['revenue_adj']==tmdb_df_cleaned['revenue_adj'].min()]

In [ ]:
tmdb_df_cleaned['revenue_adj'].nlargest(10)

In [ ]:
tmdb_top_10_revenue=tmdb_df_cleaned[lambda x :x['revenue_adj'] >= 1385748801]
tmdb_top_10_revenue.head(2)

In [ ]:
tmdb_df_cleaned['revenue_adj'].nsmallest(10)

In [ ]:
tmdb_min_10_revenue=tmdb_df_cleaned[lambda x :x['revenue_adj'] <= 6310]
tmdb_min_10_revenue

In [ ]:
#Function to plot relation between x , y
def plot_relation_rate(stage,xlabel,ylabel,message):
    type(stage)
    stage=stage.sort_values()
    plt.figure(figsize=(14, 9))
    plt.barh(stage.index,stage.array)
    plt.xlabel(xlabel, fontsize = 14)
    plt.title('relation between {} and {} {}'.format(xlabel,ylabel,message), fontsize = 16)
    #plt.gca().invert_yaxis()
    plt.show();

In [ ]:
top_stage=tmdb_top_10_revenue.groupby(['original_title'])['revenue_adj'].sum()
top_stage.astype('int64')
plot_relation_rate(top_stage,'Revenu','Movie',"top 10")

In [ ]:
top_stage=tmdb_top_10_revenue.groupby(['director'])['revenue_adj'].sum()
top_stage.astype('int64')
plot_relation_rate(top_stage,'Revenu','Director',"TOP 10")

In [ ]:
top_stage=tmdb_min_10_revenue.groupby(['director'])['revenue_adj'].sum()
top_stage.astype('int64')
plot_relation_rate(top_stage,'Revenu','Director','Lowest 10')

In [ ]:
top_stage=tmdb_min_10_revenue.groupby(['original_title'])['revenue_adj'].sum()
top_stage.astype('int64')
plot_relation_rate(top_stage,'Revenu','Movie',"Lowest 10")

### Research Question 2  (How many documentry movie there ?)

In [ ]:
# How many documentry movie What about those movies profit and compare with all movies profit
tmdb_df=tmdb_df_cleaned[tmdb_df_cleaned['genres'].str.contains('Documentary')]
tmdb_df['profit']=tmdb_df['revenue_adj']-tmdb_df['budget_adj']
tmdb_df

In [ ]:
# there is about 35 documentary movie and the relation between profit and movies
top_stage=tmdb_df.groupby(['original_title'])['profit'].sum()
top_stage.astype('int64')
plot_relation_rate(top_stage,'Profit','Documentry',"Movie")

In [ ]:
# calculate the profit for all  movies
tmdb_df_cleaned['profit']=tmdb_df_cleaned['revenue_adj']-tmdb_df_cleaned['budget_adj']

In [ ]:
#compare the precentage of losing movies budget>revenue  in both all movies and documentry movies
fail_d_movies=tmdb_df[tmdb_df['profit']<=0].count().max()
all_d_movies=tmdb_df['profit'].count().max()
percentage_fail_documentry_movies=(fail_d_movies/all_d_movies)*100
percentage_fail_all_movies=((tmdb_df_cleaned[tmdb_df_cleaned['profit']<=0].count().max())/(tmdb_df_cleaned['profit'].count().max()))*100
plt.figure(figsize=(14, 9))
plt.barh(['percentage_fail_documentry_movies','percentage_fail_all_movies'],[percentage_fail_documentry_movies,percentage_fail_all_movies])
plt.title('Relation between losing Documentry movies and all movies', fontsize = 16)
plt.show();

In [ ]:
tmdb_df_cleaned.to_csv("cleaned_tmdb.csv",index=0)

### Research Question 2  (Which cast , geners ,director has highest profit?)

In [ ]:
#function to count each one and his occurance
def data_occurance_rate_due_profit(data):
    temparr={}
    temp=data
    for item in temp:
        try:
            data_temp=re.search(r'[\w\W]+',item).group().split('|')
           # print(data_temp)
            for item in data_temp:
                if(item in temparr):
                    temparr[item]=temparr[item]+1
                else:
                    temparr[item]=1
        except :
            print('message',item)
            continue
    return temparr

In [ ]:
d=data_occurance_rate_due_profit(tmdb_df_cleaned['genres'])

In [ ]:
type(d)

In [ ]:
d

<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!